### Environment Setup
#### Libraries Used:
Protobuf 2.6
Pillow 1.0
lxml
tf Slim (which is included in the "tensorflow/models" checkout)
Jupyter notebook
Matplotlib
Tensorflow
Keras
scikit-image
scikit-learn
h5py
numpy
opencv-python
unzip


Install tensorflow object detection api following the instructions [here](https://github.com/tensorflow/models/blob/3bf85a4eddb9c56a28cc266ee4aa5604fb4d8334/object_detection/g3doc/installation.md).


To setup GPU compute instance in google cloud, I followed the instructions [here](https://medium.com/google-cloud/running-jupyter-notebooks-on-gpu-on-google-cloud-d44f57d22dbd)


Folder Structure:
 |-data
  |  |-1000
  |  |-annotated_images
  |  |-labeled_images
  |  |  |-non-vehicles
  |  |  |  |-Extras
  |  |  |  |-GTI
  |  |  |-vehicles
  |  |  |  |-GTI_Far
  |  |  |  |-GTI_Left
  |  |  |  |-GTI_MiddleClose
  |  |  |  |-GTI_Right
  |  |  |  |-KITTI_extracted
  |  |-videos
  |-models
  |  |-densenet
  |  |-faster_rcnn
  |  |  |-eval
  |  |  |-output
  |  |  |-train
  |-tf_models

### Object Detection Using Faster R_CNN


#### Create TF Records 
Tensorflow accepts inputs in the form of tf records. [create_tf_record.py] takes the inputs from _'data/annotated_images'_ and converts them to tf_records. 
_'data/annotated_images'_ has the labels and annotated images that are retrieved from [CROWDAI](https://github.com/udacity/self-driving-car/tree/master/annotations)[download link](:http://bit.ly/udacity-annoations-crowdai). Labels.csv retrieved from download link has wrong column order which causes an error. So the column order need to be changed to x_min, y_min, x_max, y_max, Frame, Label, Preview URL. 

The labels.csv has 3 classes: Car, Truck and Pedestrian. I removed the entries for Pedestrian as my goal is to detect vehicles. I also randomly shuffle the inputs. In the code you can select the number of samples you want to include in for training and validation. I chose 8000 training samples and 2000 validation samples.

The output of the code is _data/val.record_ and _data/train.record_



#### Finetune object detection Model
Training an object detection model from scratch requires huge computational resource and time. Therefore, I used the already trained model in [Tensorflow Detection Model Zoo](https://github.com/tensorflow/models/blob/3bf85a4eddb9c56a28cc266ee4aa5604fb4d8334/object_detection/g3doc/detection_model_zoo.md) I picked faster_rcnn_inception_resnet_v2_atrous_coco as it has the highest mean average precision score. 


In [ ]:
cd models/faster_rcnn

wget  http://download.tensorflow.org/models/object_detection/faster_rcnn_inception_resnet_v2_atrous_coco_11_06_2017.tar.gz

untar -xvzf faster_rcnn_inception_resnet_v2_atrous_coco_11_06_2017.tar.gz

Tensorflow accepts class labels in json format as pbtxt file. Hence, I created _data/label_map.pbtxt as per the instructions [here](https://github.com/tensorflow/models/blob/3bf85a4eddb9c56a28cc266ee4aa5604fb4d8334/object_detection/g3doc/using_your_own_dataset.md)

Tensorflow Object Detection API requires config file. I used the samples provided by [here](https://github.com/tensorflow/models/blob/3bf85a4eddb9c56a28cc266ee4aa5604fb4d8334/object_detection/samples/configs/faster_rcnn_inception_resnet_v2_atrous_pets.config).  I changed the following parameters:

- num_classes: 2
- fine_tune_checkpoint: "/home/tuncer/machine-learning/projects/capstone/models/faster_rcnn/faster_rcnn_inception_resnet_v2_atrous_coco_11_06_2017/model.ckpt"

train_input_reader: {
  tf_record_input_reader {
    input_path: "/home/tuncer/machine-learning/projects/capstone/data/train.record"
  }
  label_map_path: "/home/tuncer/machine-learning/projects/capstone/data/label_map.pbtxt"
}

eval_input_reader: {
  tf_record_input_reader {
    input_path: "/home/tuncer/machine-learning/projects/capstone/data/val.record"
  }
  label_map_path: "/home/tuncer/machine-learning/projects/capstone/data/label_map.pbtxt"
  shuffle: false
  num_readers: 1
}

My configuration file is models/faster_rcnn/faster_rcnn_gpu.config.


Run the following command to train your model from the main directory i.e., ../capstone/.[Full set of instructions](https://github.com/tensorflow/models/blob/3bf85a4eddb9c56a28cc266ee4aa5604fb4d8334/object_detection/g3doc/running_locally.md) 

In [ ]:
python tf_models/object_detection/train.py --logtostderr --pipeline_config_path=models/faster_rcnn/faster_rcnn_gpu.config --train_dir=models/faster_rcnn/train/

After running ~3084 steps, you must see following files in models/faster_rcnn:
-rw-rw-r-- 1 tuncer tuncer 453M Sep 22 02:37 model.ckpt-3084.data-00000-of-00001
-rw-rw-r-- 1 tuncer tuncer  66K Sep 22 02:37 model.ckpt-3084.index
-rw-rw-r-- 1 tuncer tuncer  20M Sep 22 02:38 model.ckpt-3084.meta


#### Saving Finetuned Model
We need to export a graph to use for inference phase at _inference.ipynb_ I exported graph by running:

In [ ]:
python tf_models/object_detection/export_inference_graph.py --input_type image_tensor --pipeline_config_path models/faster_rcnn/faster_rcnn_gpu.config  --trained_checkpoint_prefix models/faster_rcnn/train/model.ckpt-3084 --output_directory models/faster_rcnn/output

After successfull execution of the command, you should see the following files in models/faster_rcnn/outpu:
tuncer@ins1:~/machine-learning/projects/capstone$ ls -lh models/faster_rcnn/output/
total 465M
-rw-rw-r-- 1 tuncer tuncer   77 Sep 22 02:45 checkpoint
-rw-rw-r-- 1 tuncer tuncer 231M Sep 22 02:45 frozen_inference_graph.pb
-rw-rw-r-- 1 tuncer tuncer 227M Sep 22 02:45 model.ckpt.data-00000-of-00001
-rw-rw-r-- 1 tuncer tuncer  42K Sep 22 02:45 model.ckpt.index
-rw-rw-r-- 1 tuncer tuncer 7.2M Sep 22 02:45 model.ckpt.meta
drwxr-xr-x 3 tuncer tuncer 4.0K Sep 22 02:46 saved_model

### Object Classification using DenseNet

#### Input Data Creation
I used the data for [vehicle](https://s3.amazonaws.com/udacity-sdc/Vehicle_Tracking/vehicles.zip) and [non-vehicle](https://s3.amazonaws.com/udacity-sdc/Vehicle_Tracking/non-vehicles.zip) which are retrieved by Udacity from  [GTI vehicle image database](http://www.gti.ssr.upm.es/data/Vehicle_database.html), [the KITTI vision benchmark suite](http://www.cvlibs.net/datasets/kitti/).


In [ ]:
cd data/labeled_images
wget https://s3.amazonaws.com/udacity-sdc/Vehicle_Tracking/vehicles.zip
wget https://s3.amazonaws.com/udacity-sdc/Vehicle_Tracking/non-vehicles.zip

unzip vehicles.zip
unzip non-vehicles.zip

_load_data_ method in densenet.py reads all the images into numpy array. CV2 reads the images in BGR format. Images are resized to 224x224.
Mean pixel of the images are subtracted to make the dataset compatible with the pretrained models:
x[:, :, :, 0] -= 103.939
x[:, :, :, 1] -= 116.779
x[:, :, :, 2] -= 123.68

All the images are shuffled. %70 of them used for training %30 used for validation.

My classifier is binary classifier, meaning identifies if object is a vehicle or not. The number of classes is set to two. Non-vechicle images got class 0 while vehicle images got class 1 tag.


#### Train Object Classification Model
I used the [pre-trained model](https://drive.google.com/open?id=0Byy2AcGyEVxfUDZwVjU2cFNidTA) by [@flyyufelix](https://github.com/flyyufelix/DenseNet-Keras) Saved densenet161_weights_tf.h5 into model/densenet/
Run the following command to finetune DenseNet model

In [ ]:
python densenet.py

The program stores the trained model into models/densenet/densenet_finetuned_weights.h5.